In [23]:
import numpy as np
import pandas as pd
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler, OrdinalEncoder, FunctionTransformer, Normalizer
from sklearn.linear_model import LogisticRegression

In [2]:
df = pd.read_csv('train.csv')
dfout = df['Survived']

# column transformer planning

to drop: 
- passengerId
- Survived
- Name
- ticket

One hot encoding:
- sex
- embarked

convert NaN to 0:
- Cabin

Normalize:
- Age
- Fare

In [6]:
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [17]:
def NaNRemover(X):
    if isinstance(X, pd.DataFrame):
        return X['Cabin'].fillna('0')
    else:
        return np.where(pd.isnull(X), '0', X)

# Construct the column trasnformer

In [29]:
def NaNFiller(X):
    if isinstance(X, pd.DataFrame):
        X['Age'] = X['Age'].fillna(X['Age'].mean())
    elif isinstance(X, np.ndarray):
        # Convert to DataFrame to use pandas fillna
        df = pd.DataFrame(X, columns=['Age'] if X.shape[1] == 1 else None)
        df['Age'] = df['Age'].fillna(df['Age'].mean())
        X = df.to_numpy()
    return X

def NaNRemover(X):
    if isinstance(X, pd.DataFrame):
        return X['Cabin'].fillna('0')
    else:
        return np.where(pd.isnull(X), '0', X)

ct = ColumnTransformer(
    transformers=[
        ('ohe', OneHotEncoder(), ['Sex', 'Embarked']),
        ('nrem', FunctionTransformer(NaNRemover), ['Cabin']),
        ('nmean', FunctionTransformer(NaNFiller), ['Age']),
        ('minmax', MinMaxScaler(feature_range=(1,2)), ['Age']),
    ],
    remainder='drop'
)

In [30]:
pipe = Pipeline([
    ('preprocessing', ct),
    ('clf', LogisticRegression())
])

In [31]:
df = df.drop('Survived', axis=1)

KeyError: "['Survived'] not found in axis"

In [32]:
trained_pipe = pipe.fit(df, dfout)

ValueError: The output of the 'nrem' transformer should be 2D (numpy array, scipy sparse array, dataframe).

In [ ]:
from sklearn.model_selection import train_test_split

xtrain, xtest, ytrain, ytest = train_test_split()